In [2]:
from scripts.data_util.sqlConnector import online_sql_connector , update_connector
update_connector(path = './data/DB_3rdPartySQL.h5' , connector = online_sql_connector() , trace = 1)


haitong/hf_factors
since 20231219 to 20231222, total 1 periods(Q)
Fri Dec 22 00:40:21 2023 : haitong:hf_factors:202312                    

haitong/dl_factors
since 20231219 to 20231222, total 1 periods(Q)


Exception during reset or similar
Traceback (most recent call last):
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pymysql/connections.py", line 803, in _write_bytes
    self._sock.sendall(data)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _finalize_fairy
    fairy._reset(
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1438, in _reset
    pool._dialect.do_rollback(self)
  File "/home/mengkjin/.local/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 692, in do_rollback
    dbapi_connection.rollback()
  File "/home/mengkjin/.local/lib/python3.10/site-packages/pymysql/connections.py", line 487, in rollback
    self._execute_command(COMMAND.COM_QUERY, "ROLLBACK")
  File "/home/mengkjin/.local/lib/python3.10

Fri Dec 22 00:41:05 2023 : haitong:dl_factors:202312                    

dongfang/hfq_chars
since 20231219 to 20231222, total 1 periods(Q)
Fri Dec 22 00:41:12 2023 : dongfang:hfq_chars:202312                    

dongfang/l2_chars
since 20231219 to 20231222, total 1 periods(Q)
Fri Dec 22 00:41:35 2023 : dongfang:l2_chars:202312                    

dongfang/ms_chars
since 20231219 to 20231222, total 1 periods(Q)
Fri Dec 22 00:42:05 2023 : dongfang:ms_chars:202312                    

dongfang/order_flow
since 20231219 to 20231222, total 1 periods(Q)
Fri Dec 22 00:42:08 2023 : dongfang:order_flow:202312                    

dongfang/gp
since 20231216 to 20231222, total 1 periods(Q)
Fri Dec 22 00:42:10 2023 : dongfang:gp:202312                    

dongfang/tra
since 20231216 to 20231222, total 1 periods(Q)
Fri Dec 22 00:42:12 2023 : dongfang:tra:202312                    

kaiyuan/positive
since 20231101 to 20231222, total 1 periods(Q)
Fri Dec 22 00:42:17 2023 : kaiyuan:positive:202311

In [5]:
from scripts.data_util.DataTank import DataTank , Data1D
dtank = DataTank('./data/DB_3rdPartySQL.h5' , open=True , mode = 'r')
g = dtank.get_object('haitong/hf_factors/20231213')
g

<HDF5 group "/haitong/hf_factors/20231213" (3 members)>